In [1]:
import h2o
import matplotlib as plot
%matplotlib inline

In [25]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 47 mins
H2O cluster timezone:,Europe/London
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,29 days
H2O cluster name:,H2O_from_python_pasilvacorista_78yn6r
H2O cluster total nodes:,1
H2O cluster free memory:,1.579 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [26]:
url="http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip"
data = h2o.import_file(url) 

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [27]:
ignoreFields= ['ArrDelay','DepDelay','CarrierDelay','WeatherDelay', 'SecurityDelay','LateAircraftDelay','IsArrDelayed',
               'IsDepDelayed','ActualElapsedTime', 'ArrTime']

In [28]:
training_columns = [n for n in data.names if n not in ignoreFields]
response_column= "IsArrDelayed"

In [2]:
train, val, test = data.split_frame(ratios=[0.8,0.1], seed = 123)

NameError: name 'data' is not defined

In [30]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [31]:
training_columns_2 = list(filter(lambda x: x != "TailNum", training_columns ))

In [32]:
m_def = H2ODeepLearningEstimator()
%time m_def.train(x=training_columns_2, y= response_column, training_frame= train, validation_frame =val )

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Wall time: 39.6 s


In [35]:
search_criteria = { 'strategy': 'RandomDiscrete', 'seed':77,'max_models' : 12}

hyper_params={'activation': ["RectifierWithDropout"],

"input_dropout_ratio" : [0,0.05, 0.1],

'l1': [0, 0.00001,0.000001,0.000003],

'l2': [0, 0.00001,0.000001,0.000003],

#'input_dropout_ratio' : [0, 0.1, 0.2,0.3],

'hidden_dropout_ratios':[[0, 0], [0.2,0.2],[0.4,0.4],[0.6,0.6 ]]

}

In [36]:
g = h2o.grid.H2OGridSearch(model=H2ODeepLearningEstimator(epochs = 3, hidden=[400,200]),

grid_id='grid_test',

hyper_params= hyper_params,

search_criteria =search_criteria

)

In [37]:
%time g.train(x=training_columns_2, y= response_column, training_frame= train, validation_frame =val)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%
Wall time: 4min 43s


In [40]:
g.model_performance(test)


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.14926175338525197
RMSE: 0.3863440867740206
LogLoss: 0.4458749064634368
Mean Per-Class Error: 0.20063871887142581
AUC: 0.8905730015686033
pr_auc: 0.834444963091522
Gini: 0.7811460031372066
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2967532882911899: 


,NO,YES,Error,Rate
NO,1355.0,595.0,0.3051,(595.0/1950.0)
YES,306.0,2195.0,0.1224,(306.0/2501.0)
Total,1661.0,2790.0,0.2024,(901.0/4451.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2967533,0.8297108,269.0
max f2,0.1010119,0.9047936,344.0
max f0point5,0.4620491,0.8431967,197.0
max accuracy,0.3699914,0.8011683,238.0
max precision,0.9998733,1.0,0.0
max recall,0.0043395,1.0,398.0
max specificity,0.9998733,1.0,0.0
max absolute_mcc,0.3699914,0.5970205,238.0
max min_per_class_accuracy,0.3840381,0.7938462,232.0
max mean_per_class_accuracy,0.4245196,0.7993613,214.0


Gains/Lift Table: Avg response rate: 56.19 %, avg score: 45.76 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3549512,0.7442008,310.0
max f2,0.1628776,0.8655776,394.0
max f0point5,0.4816736,0.7005206,225.0
max accuracy,0.4729826,0.6672658,233.0
max precision,0.9998231,1.0,0.0
max recall,0.1628776,1.0,394.0
max specificity,0.9998231,1.0,0.0
max absolute_mcc,0.5348102,0.3189928,181.0
max min_per_class_accuracy,0.4955016,0.6543590,212.0
max mean_per_class_accuracy,0.4816736,0.6581481,225.0


Gains/Lift Table: Avg response rate: 56.19 %, avg score: 52.68 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0101101,0.9999269,1.7796881,1.7796881,1.0,0.9999592,1.0,0.9999592,0.0179928,0.0179928,77.9688125,77.9688125
,2,0.0202202,0.9998422,1.7796881,1.7796881,1.0,0.9998936,1.0,0.9999264,0.0179928,0.0359856,77.9688125,77.9688125
,3,0.0301056,0.9992985,1.7796881,1.7796881,1.0,0.9996702,1.0,0.9998423,0.0175930,0.0535786,77.9688125,77.9688125
,4,0.0402157,0.9967507,1.7796881,1.7796881,1.0,0.9982672,1.0,0.9994463,0.0179928,0.0715714,77.9688125,77.9688125
,5,0.0501011,0.9757014,1.7796881,1.7796881,1.0,0.9891638,1.0,0.9974175,0.0175930,0.0891643,77.9688125,77.9688125
,6,0.1002022,0.8189223,1.6280555,1.7038718,0.9147982,0.8837488,0.9573991,0.9405831,0.0815674,0.1707317,62.8055504,70.3871815
,7,0.1500786,0.7185386,1.4429904,1.6171717,0.8108108,0.7745386,0.9086826,0.8854007,0.0719712,0.2427029,44.2990371,61.7171694
,8,0.2001797,0.6178665,1.2609449,1.5280151,0.7085202,0.6565532,0.8585859,0.8281246,0.0631747,0.3058776,26.0944949,52.8015057
,9,0.3001573,0.5567146,1.1637961,1.4066996,0.6539326,0.5834741,0.7904192,0.7466355,0.1163535,0.4222311,16.3796055,40.6699596
,10,0.4001348,0.5236240,1.1517982,1.3430100,0.6471910,0.5386557,0.7546322,0.6946697,0.1151539,0.5373850,15.1798157,34.3010017


{'grid_test_model_10': ,
 'grid_test_model_11': ,
 'grid_test_model_3': ,
 'grid_test_model_1': ,
 'grid_test_model_8': ,
 'grid_test_model_2': ,
 'grid_test_model_5': ,
 'grid_test_model_7': ,
 'grid_test_model_6': ,
 'grid_test_model_4': ,
 'grid_test_model_12': ,
 'grid_test_model_9': }

In [47]:
g.grid_id

'grid_test'